# Zagadki

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających. 

Za to zadanie możesz zdobyć pomiędzy 0 i 1.5 punktu. Zdobędziesz 0 punktów jeśli wartość kryterium `mean reciprocal rank` na zbiorze testowym wyniesie poniżej 0.02, a 1.5 punktu jeśli wyniesie powyżej 0.3. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością kryterium.

# Kod startowy

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [3]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm
# ZMIENIOINE
import torch
import gc
import numpy as np
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
from transformers import logging
from pandas import Series

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.set_verbosity_error()
gc.collect()

557

## Ładowanie danych

In [4]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = './data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)
    if ret:
        return ret
    return word

In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/patryk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### tworzenie bases

In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

#### ladowanie modeli

In [8]:
# ZMIENIONE
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model_word2vec = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

model_name_pauga = 'flax-community/papuGaPT2'
tokenizer_papuga = AutoTokenizer.from_pretrained(model_name_pauga)
model_papuga = AutoModelForCausalLM.from_pretrained(model_name_pauga).to(device)
emb_papuga = model_papuga.transformer.wte.weight.detach().cpu().numpy()
del model_papuga

# model_name_polka = 'eryk-mazus/polka-1.1b'
# tokenizer_polka = AutoTokenizer.from_pretrained(model_name_polka)
# model_polka = AutoModelForCausalLM.from_pretrained(model_name_polka).to(device)

model_name_herbert = "allegro/herbert-base-cased"
tokenizer_herbert = AutoTokenizer.from_pretrained(model_name_herbert)
model_herbert = AutoModel.from_pretrained(model_name_herbert).to(device)

In [9]:
gc.collect()

8

#### tworzenie all_word_definitions i base_idf

In [10]:
# ZMIENONE
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
punctuation = {'!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '``', "''"}
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    L = word.split()
    if len(L) == 1:
        word = L[0]
        definition = list(filter(lambda x : x not in punctuation, tokenize(definition.lower()))) # tutaj byl set
        if len(definition) == 0: continue
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

In [11]:
Series(base_idf).describe()

count    18259.000000
mean         8.098898
std          1.130758
min          0.785740
25%          7.611471
50%          8.304619
75%          8.997766
max          8.997766
dtype: float64

#### tworzenie answers i queries

In [101]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()                  
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

In [102]:
queries[0]

['rękopiśmienny',
 'tekst',
 'lub',
 'dokument',
 ',',
 'niepublikowany',
 'drukiem',
 '.']

## Kod z kryteriami oceniającymi

In [103]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)
    
    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)
    
    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [104]:
def cos(a, b):
    return a.dot(b) / (a.dot(a) * b.dot(b)) ** 0.5

In [105]:
# def word2vec_embedding(word):
#     return model_word2vec.wv[word]

### PAPUGA i IDF
#### tworzenie all_ward_embeddings - PAPUGA i IDF

In [121]:
def decoder_embedding(word):
    token_ids = tokenizer_papuga.encode(' ' + word)
    # emb_papuga[token_ids[0]] *= 3.0
    whole_word_embbeding = np.sum(np.stack([emb_papuga[id] for id in token_ids]), axis=0)
    # whole_word_embbeding = np.sum(np.stack([emb_papuga[id] for id in token_ids]), axis=0) / len(token_ids)
    return whole_word_embbeding

In [122]:
all_word_definitions_embeddings = dd(list)

for word in tqdm(all_word_definitions, total=len(all_word_definitions)):
    for definition in all_word_definitions[word]:
        definition = list(definition)
        first_w_in_definition = definition[0]
        base = get_word_base(first_w_in_definition)
        res_embbeding = decoder_embedding(first_w_in_definition) * -base_idf[get_word_base(base)]
        # res_embbeding = decoder_embedding(first_w_in_definition)
        for w in definition[1:]:
            base = get_word_base(w)
            res_embbeding += decoder_embedding(w) * -base_idf[base]
            # res_embbeding += decoder_embedding(w)
        all_word_definitions_embeddings[word].append(res_embbeding)

for query, answer in tqdm(zip(queries[293:], answers[293:]), total=len(queries[293:])):
    guery = list(filter(lambda x : x not in punctuation, query)) # tutaj byl set
    first_w_in_definition = query[0]
    base = get_word_base(first_w_in_definition)
    res_embbeding = decoder_embedding(first_w_in_definition) * -base_idf[get_word_base(base)]
    # res_embbeding = decoder_embedding(first_w_in_definition)
    for w in query[1:]:
        base = get_word_base(w)
        res_embbeding += decoder_embedding(w) * -base_idf[base]
        # res_embbeding += decoder_embedding(w)
    all_word_definitions_embeddings[answer].append(res_embbeding)

  0%|          | 0/8085 [00:00<?, ?it/s]

100%|██████████| 1700/1700 [00:02<00:00, 606.24it/s]


In [123]:
def answer_riddle(riddle, K):
    # papuga & idf & earest neighbor word2vec
    riddle = list(filter(lambda x: x not in punctuation, riddle))
    res_embbeding = decoder_embedding(riddle[0]) * -base_idf[get_word_base(riddle[0])]
    # res_embbeding = decoder_embedding(riddle[0])
    for word in riddle[1:]:
        res_embbeding += decoder_embedding(word) * -base_idf[get_word_base(word)]
        # res_embbeding += decoder_embedding(word)
    
    closest_words = []
    for word, embeddings in all_word_definitions_embeddings.items():
        for emb in embeddings:
            # distance = np.linalg.norm(res_embbeding - emb)
            distance = cos(res_embbeding, emb)
            closest_words.append((distance, word))
    
    closest_words = sorted(closest_words, key=lambda x: x[0])
    return [word for _, word in closest_words[-K:]]

### HERBERT
#### tworzenie all_word_definitions_embeddings - HERBERT

In [112]:
def encoder_embedding(L):
    txt = ' '.join(L)
    input_ids = tokenizer_herbert(txt, return_tensors='pt')['input_ids'].to(device)
    output = model_herbert(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

In [114]:
all_word_definitions_embeddings = dd(list)

for word in tqdm(all_word_definitions):
    for definition in all_word_definitions[word]:
        definition = list(filter(lambda x: base_idf[x] > 2.9, definition))
        def_embbeding = encoder_embedding(definition)
        all_word_definitions_embeddings[word].append(def_embbeding)
        
for query, answer in tqdm(zip(queries[293:], answers[293:]), total=len(queries[293:])):
    guery = list(filter(lambda x : x not in punctuation and base_idf[x] > 2.9, query)) # tutaj byl set
    query_embbeding = encoder_embedding(query)
    all_word_definitions_embeddings[answer].append(query_embbeding)

100%|██████████| 1700/1700 [00:23<00:00, 71.46it/s]


In [115]:
def answer_riddle(riddle, K):
    riddle = list(filter(lambda x: x not in punctuation and base_idf[x] > 2.9, riddle))
    riddle_embbeding = encoder_embedding(riddle)

    closest_words = []
    for word, embeddings in all_word_definitions_embeddings.items():
        for emb in embeddings:
            # distance = np.linalg.norm(res_embbeding - emb)
            distance = cos(riddle_embbeding, emb)
            closest_words.append((distance, word))
    
    closest_words = sorted(closest_words, key=lambda x: x[0])
    return [word for _, word in closest_words[-K:]]

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [124]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(set(query), K=K))
    score = mean_reciprocal_rank(answers, computed_answers, K=K)
    
    return score

In [125]:
gc.collect()

0

In [126]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    PART_OF_DATA = 293
    K = 20
    valid_queries = queries[:PART_OF_DATA]
    valid_answers = answers[:PART_OF_DATA]
    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered: 100%|██████████| 293/293 [00:59<00:00,  4.89it/s]

Mean Reciprocal Rank = 0.020161534304620136
Score: 0.020161534304620136


 herbert \
 queries answered: 100%|██████████| 293/293 [01:04<00:00,  4.51it/s] \
 Mean Reciprocal Rank = 0.0036328821908762523 \
Score: 0.0036328821908762523 



herbert dla idf > 2.9 \
queries answered: 100%|██████████| 293/293 [01:04<00:00,  4.51it/s] \
Mean Reciprocal Rank = 0.0017245330234689801 \
Score: 0.0017245330234689801



papgua suma embedow pojedyczych wyrazow \
queries answered: 100%|██████████| 293/293 [01:01<00:00,  4.80it/s] \
Mean Reciprocal Rank = 0.018473519372329588 \
Score: 0.018473519372329588



papuga suma ebedow slow wazona  -idf \
queries answered: 100%|██████████| 293/293 [01:01<00:00,  4.74it/s] \
Mean Reciprocal Rank = 0.020161534304620136 \
Score: 0.020161534304620136

